## Data sourcing

Source data from various source systems and ingest them using python code.

1. Parquet files
2. CSV files
3. APIs
4. RDBMS databases
5. HTML
6. Text
7. XML
8. Excel
9. GeoJSON

https://pandas.pydata.org/pandas-docs/dev/user_guide/io.html

In [ ]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
from unicodedata import normalize

from project.kode.hand_on_practice2.Pengenalan_pipeline import pipeline_baca_data

### Sourcing Parquet data

Please visit the url https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
# Read data from the Parquet file. We use pandas read_parquet method for ease and speed.
df_parquet = pd.read_parquet("yellow_tripdata_2022-01.parquet")
df_parquet.head()

### Sourcing CSV data 

Please visit the url https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=500


In [ ]:
# Read data from the CSV file. We use pandas read_csv method for ease and speed.
df_csv = pd.read_csv("h9gi-nx95.csv")
df_csv.head()

### Sourcing data from APIs

Please make sure to install the certifi library using - pipenv install certifi

In [ ]:
# get api data from url
url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500'

# Check if API is available to retrive the data
http = urllib3.PoolManager()
apt_status = http.request('GET', url).status
print(apt_status)
if apt_status == 200:
    # Sometimes we get certificate error . We shoul never silence this error as this may cause a securirty threat.
    # Create a Pool manager that can be used to read the API response 
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    data = json.loads(http.request('GET', url).data.decode('utf-8'))
    df_api = pd.json_normalize(data)
else:
    df_api = pd.Dataframe()
df_api.head(10)

### Sourcing Data from RDBMS tables

In [ ]:
# Read sqlite query results into a pandas DataFrame
with sqlite3.connect("movies.sqlite") as conn:
    df = pd.read_sql("SELECT * from movies", conn)
df.head()

# Sourcing data from Webpages

Please visit the url https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)

In [ ]:
# get data from url
df_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)',match = 'by country')
# Let's see how many tables are there with tage ' by county'
print(len(df_html)) # There are 4 tables
# Let's see the first table
df_html[0]

### Sourcing Data from GeoJSON
- https://github.com/9uz/wilayah-indonesia/tree/master/data/geojson/regency
- https://github.com/9uz/indonesia-district

In [ ]:
import json
uri="https://raw.githubusercontent.com/9uz/indonesia-district/refs/heads/master/id35_jawa_timur/id3577_kota_madiun/id3577_kota_madiun.geojson"

import geopandas

df = geopandas.read_file(uri)
zone = df.dissolve(by='district')
display(zone)
zone['geometry'].plot()

# Sourcing Data from PDF



In [27]:
# !pip install pypdf
# Fungsi ekstrak data dari pdf
from pypdf import PdfReader
import urllib3
from urllib3 import request
from io import BytesIO

def dowloadFile():
    url = "https://disdukcapil.surabaya.go.id/wp-content/uploads/2022/11/Proyeksi-Penduduk-2023-2032.pdf"
    http = urllib3.PoolManager()
    wFile = http.request("GET", url)
    bytes_stream = BytesIO(wFile.data)  # Mengonversi data file menjadi stream byte
    return bytes_stream

def getText(bytes_stream):
  text = ""
  try :
    # bytes_stream Bisa diisi lokasi file atau stream dari url
    reader = PdfReader(bytes_stream,strict=False)
    pages = reader.pages
    for page in pages:
      text += page.extract_text()
    #print(text)
  except :
     text =""
  return text

f=dowloadFile()
text = getText(f)
text

# Sourcing Data from MAPS

In [30]:
!pip install geemap

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 2.3/2.3 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 1.2/1.2 MB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 14.9/14.9 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 10.8/10.8 MB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 1.3/1.3 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 7.5/7.5 MB 11.6 MB/s eta 0:00:00
   ---------------------------------------- 14.8/14.8 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 1.2/1.2 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 1.5/1

In [31]:
import geemap
import ee
import pandas as pd

## sesuaikan judul proyek di google eart engine
cloud_project = 'ee-123456'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

KeyboardInterrupt: Interrupted by user

In [ ]:
def initialize_map(admin_name='Madiun', province='Jawa Timur'):
    """Initialize map and filter administrative boundaries."""
    m = geemap.Map()
    admin2 = ee.FeatureCollection("FAO/GAUL/2015/level2")
    filtered_admin2 = admin2 \
        .filter(ee.Filter.eq('ADM2_NAME', admin_name)) \
        .filter(ee.Filter.eq('ADM1_NAME', province))
    geometry = filtered_admin2.geometry()
    m.centerObject(geometry, 10)
    return m, geometry

def get_sentinel_imagery(geometry, start_date, end_date, cloud_threshold=30):
    """Get and process Sentinel-2 imagery."""
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    filtered = (s2
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold))
                .filter(ee.Filter.date(start_date, end_date))
                .filter(ee.Filter.bounds(geometry)))

    # Mask low quality pixels
    def mask_low_qa(image):
        qa_band = 'cs'
        clear_threshold = 0.5
        mask = image.select(qa_band).gte(clear_threshold)
        return image.updateMask(mask)
    filtered_masked = (filtered.map(mask_low_qa)).select('B.*')

    return filtered_masked

def extract_to_dataframe(geometry, start_date, end_date, cloud_threshold=30):
    # Ambil koleksi citra berdasarkan parameter
    filtered_masked = get_sentinel_imagery(geometry, start_date, end_date, cloud_threshold)
    # Fungsi untuk mengambil data per citra
    def extract_image_data(image):
        # Ambil nilai dari beberapa band (misalnya B2, B3, B4, dst.)
        bands = ['B2', 'B3', 'B4', 'B8']
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=10,  # Resolusi spatial (m) sesuaikan dengan dataset
            maxPixels=1e8
        )

        stats = stats.getInfo()  # Mengambil data sebagai dictionary
        # Memastikan bahwa setiap band ada dalam dictionary
        return {band: stats.get(band, None) for band in bands}
    # Ekstrak data dari semua citra dan simpan ke dalam list
    data = filtered_masked.toList(filtered_masked.size()).getInfo()
    # Extract nilai band dan simpan dalam list untuk setiap citra
    extracted_data = [extract_image_data(ee.Image(item)) for item in data]
    # Convert data ke DataFrame
    df = pd.DataFrame(extracted_data)
    return df

In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'

m, geometry=initialize_map()
df = extract_to_dataframe(geometry, start_date, end_date)
print(df.head())

# Sourcing Data from GOOGLE Play

In [5]:
#!pip install google-play-scraper
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
        'id.qoin.korlantas.user',          # ID aplikasi
        lang='id',             # Bahasa ulasan (default: 'en')
        country='id',          # Negara (default: 'us')
        sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
        count=100             # Jumlah maksimum ulasan yang ingin diambil
)
type(scrapreview)

list

# Sourcing from CCTV

In [1]:
!pip install opencv-python
!pip install websockets
!pip install nest_asyncio


In [14]:
import asyncio
import websockets
import cv2
import numpy as np
import time
import datetime
import os
from IPython.display import display, HTML, clear_output

# Fungsi untuk menyimpan raw stream ke file
async def save_raw_stream(url, output_file, duration_seconds=30):
    try:
        print(f"Mencoba terhubung ke {url}...")
        # Buat folder untuk menyimpan hasil jika belum ada
        os.makedirs('video_output', exist_ok=True)
        # Siapkan file untuk menyimpan raw data
        with open(output_file, 'wb') as f:
            # Buat koneksi WebSocket
            async with websockets.connect(url) as websocket:
                print("Terhubung ke WebSocket CCTV!")
                print("Mulai mengumpulkan data video...")
                start_time = time.time()
                bytes_received = 0
                # Loop untuk menerima data
                while time.time() - start_time < duration_seconds:
                    try:
                        # Terima data dari WebSocket
                        data = await websocket.recv()
                        # Tulis data langsung ke file
                        f.write(data)
                        bytes_received += len(data)
                        if bytes_received % 100000 < len(data):  # Update setiap ~100KB
                            elapsed = time.time() - start_time
                            print(f"Menerima data: {bytes_received/1024:.2f} KB | Waktu: {elapsed:.2f} detik", end='\r')
                    except Exception as e:
                        print(f"\nError menerima data: {e}")
                        break
                print(f"\nSelesai! Total data diterima: {bytes_received/1024/1024:.2f} MB")
        return output_file
    except Exception as e:
        print(f"Error koneksi: {e}")
        return None

# Fungsi untuk mengkonversi raw stream ke format video standar menggunakan FFmpeg
def convert_to_mp4(input_file, output_file):
    try:
        # Periksa apakah FFmpeg tersedia
        import subprocess
        print("Mengkonversi stream ke MP4 menggunakan FFmpeg...")
        # Jalankan perintah FFmpeg
        command = [
            'ffmpeg',
            '-i', input_file,
            '-c:v', 'libx264',
            '-preset', 'fast',
            '-crf', '23',
            '-y',  # Timpa file jika sudah ada
            output_file
        ]

        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode == 0:
            print(f"Konversi berhasil! Video tersimpan di: {output_file}")
            return True
        else:
            print(f"Error konversi: {result.stderr}")
            return False

    except Exception as e:
        print(f"Error mengkonversi file: {e}")
        print("Anda perlu menginstal FFmpeg untuk konversi video.")
        return False

# Fungsi untuk menganalisis header dari stream
def analyze_stream_header(file_path):
    with open(file_path, 'rb') as f:
        data = f.read(1024)  # Baca 1KB pertama untuk analisis

    print("Analisis 50 byte pertama dari stream:")
    print("Hex:", ' '.join(f'{b:02x}' for b in data[:50]))
    print("ASCII:", ''.join(chr(b) if 32 <= b <= 126 else '.' for b in data[:50]))

    # Deteksi format berdasarkan header
    if data[:4] == b'GA\x00\x1e':
        print("Kemungkinan format: JSMPEG custom header")
    if b'\x00\x00\x01' in data[:20]:
        print("Kemungkinan format: MPEG Elementary Stream atau MPEG-TS")

    return data[:50]

# Fungsi utama
async def capture_cctv_stream(websocket_url, duration_seconds=30):
    # Buat nama file dengan timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    raw_file = f"video_output/cctv_raw_{timestamp}.ts"
    mp4_file = f"video_output/cctv_{timestamp}.mp4"

    print(f"Mulai merekam dari {websocket_url}")
    print(f"Raw stream akan disimpan ke {raw_file}")

    # Rekam stream mentah
    saved_file = await save_raw_stream(websocket_url, raw_file, duration_seconds)

    if saved_file:
        # Analisis header stream
        header_bytes = analyze_stream_header(saved_file)
        # Coba konversi ke MP4 menggunakan FFmpeg
        try:
            convert_success = convert_to_mp4(saved_file, mp4_file)
            if convert_success:
                return mp4_file
        except:
            print("Konversi ke MP4 gagal. Anda masih bisa menggunakan file raw untuk analisis lebih lanjut.")

        return saved_file

    return None

# Function to display HTML button
def display_download_button(filepath):
    if os.path.exists(filepath):
        file_size = os.path.getsize(filepath) / (1024 * 1024)  # Size in MB
        display(HTML(
            f'<div style="padding: 10px; background-color: #f0f0f0; border-radius: 5px;">'
            f'<p>File tersimpan: {filepath} ({file_size:.2f} MB)</p>'
            f'<button onclick="window.open(\'{filepath.replace(" ", "%20")}\', \'_blank\')">Buka File</button>'
            f'</div>'
        ))

# Jalankan di Jupyter Notebook
async def main():
    url = "wss://cctv.villabs.id/streamer-jsmpeg/streamer/pahlawan1"
    output_path = await capture_cctv_stream(url, duration_seconds=20)

    if output_path:
        display_download_button(output_path)
        print("\nUntuk mengunduh file, klik kanan pada file di panel Jupyter dan pilih 'Download'")

# Untuk menjalankan di Jupyter Notebook
# Jalankan cell ini
try:
    await main()
except:
    asyncio.run(main())

# Petunjuk tambahan untuk menjalankan tanpa FFmpeg
print("\n---- PETUNJUK TAMBAHAN ----")
print("Jika Anda tidak memiliki FFmpeg, Anda bisa memutar file .ts yang dihasilkan dengan:")
print("1. VLC Media Player: Buka file -> Media -> Open File")
print("2. ffplay (bagian dari FFmpeg): ffplay video_output/cctv_raw_*.ts")
print("3. MPV Player: mpv video_output/cctv_raw_*.ts")

Received 3196 bytes
First 20 bytes: b'GA\x00\x16\x00\x00\x01\xe0\x00\x00\x80\xc0\n1F\x97(\x01\x11F'
Format stream tidak diketahui
Received 2632 bytes
First 20 bytes: b'GA\x005\x07\x10\x08\xd2N\xf4~\x00\x00\x00\x01\xe0\x00\x00\x80\xc0'
Format stream tidak diketahui
Received 2820 bytes
First 20 bytes: b'GA\x00\x13\x00\x00\x01\xe0\x00\x00\x80\xc0\n1F\x97V\xe1\x11F'
Format stream tidak diketahui
Received 3008 bytes
First 20 bytes: b'G@\x00\x1c\x00\x00\xb0\r\x00\x01\xc1\x00\x00\x00\x01\xf0\x00*\xb1\x04'
Format stream tidak diketahui
Received 2632 bytes
First 20 bytes: b'GA\x000\x07\x10\x08\xd2`\x88~\x00\x00\x00\x01\xe0\x00\x00\x80\xc0'
Format stream tidak diketahui
Failed to decode frame.
Failed to decode frame.
Received 3008 bytes
First 20 bytes: b'GA\x00\x1e\x00\x00\x01\xe0\x00\x00\x80\xc0\n1F\x97\x9d1\x11F'
Format stream tidak diketahui
Received message with length: 3008
Mulai merekam dari wss://cctv.villabs.id/streamer-jsmpeg/streamer/pahlawan1
Raw stream akan disimpan ke video_output/c


Untuk mengunduh file, klik kanan pada file di panel Jupyter dan pilih 'Download'

---- PETUNJUK TAMBAHAN ----
Jika Anda tidak memiliki FFmpeg, Anda bisa memutar file .ts yang dihasilkan dengan:
1. VLC Media Player: Buka file -> Media -> Open File
2. ffplay (bagian dari FFmpeg): ffplay video_output/cctv_raw_*.ts
3. MPV Player: mpv video_output/cctv_raw_*.ts


In [ ]:
import asyncio
import websockets
import cv2
import numpy as np
import time
import datetime
import os
import threading
import queue
import base64
from IPython.display import display, HTML, clear_output, Image, update_display
import ipywidgets as widgets
import io
from PIL import Image as PILImage

# Antrian untuk menyimpan frame
frame_queue = queue.Queue(maxsize=100)
stop_event = threading.Event()
last_frame = None

# Fungsi untuk menerima data dari WebSocket dan memproses
async def receive_jsmpeg_data(url):
    global last_frame
    try:
        print(f"Menghubungkan ke {url}...")
        async with websockets.connect(url) as websocket:
            print("Terhubung! Mulai menerima data video...")

            # Buffer untuk menyimpan data yang diterima
            buffer = bytearray()

            # Bytes untuk menandai frame MPEG
            sequence_header = b'\x00\x00\x01\xB3'
            picture_start = b'\x00\x00\x01\x00'

            # Loop selama stop_event tidak diset
            while not stop_event.is_set():
                try:
                    # Terima data dari WebSocket
                    data = await websocket.recv()

                    # Jika 4 byte pertama adalah "JSMP", ini adalah header JSMpeg - lewati
                    if len(data) > 4 and data[0:4] == b'jsmp':
                        data = data[4:]  # Lewati header JSMpeg

                    # Tambahkan data ke buffer
                    buffer.extend(data)

                    # Cari penanda frame baru
                    idx = buffer.find(picture_start)
                    if idx != -1 and len(buffer) > idx + 1000:  # Pastikan ada cukup data
                        # Ekstrak paket antara dua penanda frame (satu frame lengkap)
                        next_idx = buffer.find(picture_start, idx + 4)
                        if next_idx != -1:
                            frame_data = buffer[idx:next_idx]
                            buffer = buffer[next_idx:]  # Simpan sisa data untuk frame berikutnya

                            # Proses frame menggunakan OpenCV (opsional)
                            try:
                                # Simpan frame ke file sementara
                                temp_file = 'temp_frame.mpg'
                                with open(temp_file, 'wb') as f:
                                    # Tambahkan header MPEG jika belum ada
                                    seq_idx = max(0, buffer.find(sequence_header, max(0, idx-100), idx))
                                    if seq_idx > 0:
                                        f.write(buffer[seq_idx:idx])
                                    f.write(frame_data)

                                # Coba baca frame dengan OpenCV
                                cap = cv2.VideoCapture(temp_file)
                                ret, frame = cap.read()
                                cap.release()

                                if ret:
                                    if not frame_queue.full():
                                        frame_queue.put(frame)
                                    last_frame = frame

                                # Hapus file sementara
                                if os.path.exists(temp_file):
                                    os.remove(temp_file)

                            except Exception as e:
                                print(f"Error memproses frame: {e}")
                                continue

                except Exception as e:
                    print(f"Error menerima data: {e}")
                    await asyncio.sleep(1)  # Jeda jika terjadi error

    except Exception as e:
        print(f"Koneksi terputus: {e}")

    finally:
        print("Selesai menerima data")

# Fungsi untuk menampilkan frame secara real-time di Jupyter
def display_frames():
    global last_frame
    output = widgets.Output()
    display(output)

    while not stop_event.is_set():
        try:
            if not frame_queue.empty():
                frame = frame_queue.get(timeout=1)
                frame_queue.task_done()

                # Konversi frame ke format RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Konversi ke format base64 untuk ditampilkan sebagai gambar
                pil_img = PILImage.fromarray(frame_rgb)
                buffer = io.BytesIO()
                pil_img.save(buffer, format="JPEG")
                img_str = base64.b64encode(buffer.getvalue()).decode()

                # Tampilkan frame
                with output:
                    clear_output(wait=True)
                    display(HTML(f'<img src="data:image/jpeg;base64,{img_str}" width="640"/>'))

            time.sleep(0.1)  # Beri waktu untuk update UI

        except queue.Empty:
            if last_frame is not None:
                # Tampilkan frame terakhir jika queue kosong
                frame_rgb = cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB)
                pil_img = PILImage.fromarray(frame_rgb)
                buffer = io.BytesIO()
                pil_img.save(buffer, format="JPEG")
                img_str = base64.b64encode(buffer.getvalue()).decode()

                with output:
                    clear_output(wait=True)
                    display(HTML(f'<img src="data:image/jpeg;base64,{img_str}" width="640"/>'))

            time.sleep(0.5)  # Beri waktu lebih lama jika tidak ada frame baru
        except Exception as e:
            print(f"Error menampilkan frame: {e}")
            time.sleep(1)

# Fungsi alternatif untuk menampilkan frame jika metode di atas tidak berhasil
def display_frames_with_ffmpeg():
    import subprocess
    import os

    # Buat folder untuk video jika belum ada
    os.makedirs('video_output', exist_ok=True)

    # Siapkan nama file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    raw_file = f"video_output/stream_{timestamp}.ts"
    html_file = f"video_output/player_{timestamp}.html"

    # Tampilkan loading spinner
    display(HTML('''
        <div style="text-align: center; margin: 20px;">
            <div style="font-size: 24px; margin-bottom: 10px;">Mempersiapkan player video...</div>
            <div style="display: inline-block; width: 40px; height: 40px; border: 5px solid #f3f3f3;
                 border-top: 5px solid #3498db; border-radius: 50%; animation: spin 2s linear infinite;">
            </div>
        </div>
        <style>
            @keyframes spin {
                0% { transform: rotate(0deg); }
                100% { transform: rotate(360deg); }
            }
        </style>
    '''))

    # Buat HTML player sederhana
    with open(html_file, 'w') as f:
        f.write(f'''
        <!DOCTYPE html>
        <html>
        <head>
            <title>CCTV Player</title>
            <style>
                body {{ margin: 0; padding: 20px; font-family: Arial, sans-serif; }}
                .container {{ text-align: center; }}
                h2 {{ color: #333; }}
            </style>
        </head>
        <body>
            <div class="container">
                <h2>Live CCTV Stream</h2>
                <video id="video" controls autoplay style="max-width: 100%; height: auto;">
                    <source src="{raw_file}" type="video/mp2t">
                    Browser Anda tidak mendukung tag video.
                </video>
            </div>
        </body>
        </html>
        ''')

    # Tampilkan link ke player HTML
    display(HTML(f'''
        <div style="padding: 15px; background-color: #f8f9fa; border-radius: 5px; margin-top: 20px;">
            <p style="font-size: 16px; margin-bottom: 10px;">Player video siap!</p>
            <a href="{html_file}" target="_blank" style="display: inline-block; padding: 10px 20px;
               background-color: #4CAF50; color: white; text-decoration: none; border-radius: 4px;">
               Buka Player Video di Tab Baru
            </a>
            <p style="font-size: 12px; color: #666; margin-top: 10px;">
                (Video akan mulai merekam ketika Anda membuka player)
            </p>
        </div>
    '''))

    # Tampilkan pesan untuk metode alternatif
    print("\nJika player video tidak berfungsi dengan baik, Anda bisa mencoba metode berikut:")
    print("1. Salin URL WebSocket: wss://cctv.villabs.id/streamer-jsmpeg/streamer/pahlawan1")
    print("2. Gunakan aplikasi seperti VLC dengan Plugin Streaming URL")
    print("   - Di VLC: Media > Open Network Stream > Masukkan URL dengan protokol yang sesuai")

# Fungsi utama
async def stream_cctv(websocket_url):
    global stop_event

    # Cek apakah IPython widgets tersedia
    try:
        import ipywidgets
        has_widgets = True
    except ImportError:
        has_widgets = False

    # Cek apakah OpenCV dan PIL tersedia
    try:
        import cv2
        import PIL
        has_cv2_pil = True
    except ImportError:
        has_cv2_pil = False

    # Reset stop event
    stop_event.clear()

    # Tampilkan opsi kontrol
    controls = widgets.HBox([
        widgets.Button(description="Stop Stream", button_style="danger"),
        widgets.Button(description="Mulai Ulang", button_style="success")
    ])
    display(controls)

    def on_stop_button_click(b):
        stop_event.set()
        print("Menghentikan stream...")

    def on_restart_button_click(b):
        stop_event.set()
        time.sleep(1)
        asyncio.create_task(stream_cctv(websocket_url))
        print("Memulai ulang stream...")

    controls.children[0].on_click(on_stop_button_click)
    controls.children[1].on_click(on_restart_button_click)

    # Coba tampilkan secara langsung jika semua dependensi tersedia
    if has_widgets and has_cv2_pil:
        print("Memulai streaming video langsung...")

        # Jalankan receiver di thread terpisah
        receiver_task = asyncio.create_task(receive_jsmpeg_data(websocket_url))

        # Jalankan displayer di thread terpisah
        display_thread = threading.Thread(target=display_frames)
        display_thread.daemon = True
        display_thread.start()

        try:
            # Tunggu hingga stop_event diset
            while not stop_event.is_set():
                await asyncio.sleep(1)
        finally:
            # Bersihkan
            stop_event.set()
            if not receiver_task.done():
                receiver_task.cancel()
            display_thread.join(timeout=2)
    else:
        # Gunakan metode fallback untuk menampilkan streaming
        print("Beberapa pustaka tidak tersedia untuk streaming langsung.")
        print("Menggunakan metode alternatif...")
        display_frames_with_ffmpeg()

# Jalankan streaming
async def main():
    url = "wss://cctv.villabs.id/streamer-jsmpeg/streamer/pahlawan1"
    print("Pastikan Anda sudah menginstal pustaka yang diperlukan:")
    print("!pip install websockets opencv-python pillow ipywidgets")

    print("\nMemulai streaming CCTV...")
    await stream_cctv(url)

# Untuk menjalankan di Jupyter Notebook
# Jalankan cell ini
try:
    await main()
except:
    asyncio.run(main())

Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Received message with length: 65536
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Pastikan Anda sudah menginstal pustaka yang diperlukan:
!pip install websockets opencv-python pillow ipywidgets

Memulai streaming CCTV...


Memulai streaming video langsung...


Output()

Menghubungkan ke wss://cctv.villabs.id/streamer-jsmpeg/streamer/pahlawan1...
Failed to decode frame.
Failed to decode frame.
Received message with length: 30720
Received message with length: 8648
Received message with length: 3196
Received message with length: 3008
Received message with length: 3572
Received message with length: 2820
Received message with length: 3196
Received message with length: 2632
Received message with length: 2256
Received message with length: 2444
Received message with length: 2068
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Failed to decode frame.
Received 65536 bytes
First 20 bytes: b'G@\x00\x15\x00\x00\xb0\r\x00\x01\xc1\x00\x00\x00\x01\xf0\x00*\xb1\x04'
Format stream tidak diketahui
Received 30720 bytes
First 20 bytes: b'H@\x9b\xd8\xca\xe7\xb6\xbf<\xe0C\x05(F\xbfB_\x03)`'
Format stream tidak diketahui
Failed to decode frame.
Failed to de

Memulai streaming video langsung...


Output()

Menghubungkan ke wss://cctv.villabs.id/streamer-jsmpeg/streamer/pahlawan1...
Koneksi WebSocket ditutup.
Koneksi WebSocket ditutup.
Koneksi WebSocket ditutup.
Terhubung! Mulai menerima data video...


In [ ]:
!pip install websockets opencv-python pillow ipywidgets

In [1]:
# Install library yang diperlukan
!pip install websocket-client opencv-python numpy pillow

import websocket
import cv2
import numpy as np
import time
import threading
import io
from datetime import datetime
from PIL import Image
from google.colab.patches import cv2_imshow
from IPython.display import clear_output, display
import base64
import json
import os
try:
  from google.colab import files
except ImportError:
  print("Google Colab tidak terdeteksi.")

class JsMpegWebSocketClient:
    def __init__(self, ws_url):
        """
        Inisialisasi WebSocket client untuk JSMpeg stream.

        Args:
            ws_url (str): URL WebSocket untuk stream CCTV (wss://)
        """
        self.ws_url = ws_url
        self.ws = None
        self.is_connected = False
        self.buffer = bytearray()
        self.current_frame = None
        self.frame_available = False
        self.lock = threading.Lock()
        self.running = False
        self.frame_count = 0
        self.start_time = time.time()

    def on_message(self, ws, message):
        """Callback ketika pesan diterima dari WebSocket."""
        with self.lock:
            # Untuk JSMpeg, kita perlu mengumpulkan data dan mem-parsing frame MPEG
            # Ini adalah pendekatan sederhana, implementasi lengkap memerlukan parser JSMpeg
            self.buffer.extend(message)

            # Coba deteksi frame JPEG dalam buffer
            # Catatan: Pendekatan ini mungkin tidak sempurna untuk semua jenis JSMpeg stream
            # tetapi bekerja untuk banyak implementasi dasar
            try:
                # Coba parse buffer sebagai frame JPEG
                image = Image.open(io.BytesIO(self.buffer))
                frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
                self.current_frame = frame
                self.frame_available = True
                self.frame_count += 1
                # Reset buffer setelah frame berhasil diproses
                self.buffer = bytearray()
            except Exception as e:
                # Jika error, kemungkinan buffer belum berisi frame lengkap
                pass

    def on_error(self, ws, error):
        """Callback ketika terjadi error pada WebSocket."""
        print(f"Error: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        """Callback ketika WebSocket ditutup."""
        print(f"WebSocket ditutup. Status: {close_status_code}, Pesan: {close_msg}")
        self.is_connected = False

    def on_open(self, ws):
        """Callback ketika WebSocket terbuka."""
        print("WebSocket terhubung!")
        self.is_connected = True

    def start(self):
        """Memulai koneksi WebSocket."""
        print(f"Mencoba menghubungkan ke {self.ws_url}...")
        # Buat WebSocket dengan callback
        websocket.enableTrace(False)
        self.ws = websocket.WebSocketApp(self.ws_url,
                                        on_open=self.on_open,
                                        on_message=self.on_message,
                                        on_error=self.on_error,
                                        on_close=self.on_close)

        # Jalankan WebSocket dalam thread terpisah
        self.running = True
        self.ws_thread = threading.Thread(target=self.ws.run_forever)
        self.ws_thread.daemon = True
        self.ws_thread.start()

    def stop(self):
        """Menghentikan koneksi WebSocket."""
        self.running = False
        if self.ws:
            self.ws.close()

    def get_frame(self):
        """Mendapatkan frame terbaru."""
        with self.lock:
            if self.frame_available:
                self.frame_available = False
                return self.current_frame.copy()
            return None

def stream_wss_cctv(ws_url, show_video=True, save_video=False, output_path=None,
                   max_frames=500, display_interval=5):
    """
    Stream data dari CCTV menggunakan WebSocket JSMpeg.

    Args:
        ws_url (str): URL WebSocket untuk stream CCTV
        show_video (bool): Menampilkan video di Colab jika True
        save_video (bool): Menyimpan video ke file jika True
        output_path (str): Path untuk menyimpan video jika save_video=True
        max_frames (int): Jumlah maksimum frame yang akan diproses (-1 untuk tanpa batas)
        display_interval (int): Interval frame untuk ditampilkan

    Returns:
        None
    """
    # Inisialisasi client WebSocket
    client = JsMpegWebSocketClient(ws_url)
    client.start()

    # Tunggu hingga terhubung
    connection_timeout = 10  # detik
    start_wait = time.time()
    while not client.is_connected and time.time() - start_wait < connection_timeout:
        time.sleep(0.5)
        print("Menunggu koneksi...", end="\r")

    if not client.is_connected:
        print(f"Tidak dapat terhubung ke {ws_url} dalam {connection_timeout} detik.")
        client.stop()
        return

    print("Koneksi berhasil dibuat. Memulai streaming...")

    # Mendapatkan frame pertama untuk informasi resolusi
    first_frame = None
    first_frame_timeout = 10  # detik
    start_wait = time.time()
    while first_frame is None and time.time() - start_wait < first_frame_timeout:
        first_frame = client.get_frame()
        if first_frame is None:
            time.sleep(0.5)
            print("Menunggu frame pertama...", end="\r")

    if first_frame is None:
        print(f"Tidak menerima frame dalam {first_frame_timeout} detik.")
        client.stop()
        return

    # Mendapatkan informasi dari frame pertama
    frame_height, frame_width = first_frame.shape[:2]
    print(f"Stream berhasil dibuka - Resolusi: {frame_width}x{frame_height}")

    # Setup video writer jika perlu menyimpan video
    video_writer = None
    if save_video:
        if output_path is None:
            # Buat nama file berdasarkan timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = f"wss_cctv_stream_{timestamp}.mp4"

        # Menggunakan codec H.264
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = 25  # Asumsi default untuk JSMpeg
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    try:
        frame_count = 0
        last_frame_time = time.time()
        start_time = time.time()

        # Main loop untuk mendapatkan dan memproses frame
        while client.running and (max_frames == -1 or frame_count < max_frames):
            # Mendapatkan frame terbaru
            frame = client.get_frame()

            if frame is not None:
                frame_count += 1
                current_time = time.time()

                # Tambahkan timestamp dan informasi frame
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                cv2.putText(frame, timestamp, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # Hitung dan tampilkan FPS
                elapsed_time = current_time - start_time
                if elapsed_time > 0:
                    fps = frame_count / elapsed_time
                    cv2.putText(frame, f"FPS: {fps:.1f}", (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                # Tampilkan frame jika diminta dan pada interval yang ditentukan
                if show_video and (frame_count % display_interval == 0):
                    cv2_imshow(frame)
                    # Hapus output sebelumnya agar tampilan tetap bersih
                    clear_output(wait=True)

                # Simpan frame jika diminta
                if save_video and video_writer is not None:
                    video_writer.write(frame)

                # Update waktu frame terakhir
                last_frame_time = current_time

            # Cek jika tidak ada frame baru dalam 5 detik
            if time.time() - last_frame_time > 5:
                print("Tidak menerima frame baru dalam 5 detik. Periksa koneksi.")

            # Sedikit delay untuk mengurangi beban CPU
            time.sleep(0.02)

    except KeyboardInterrupt:
        print("Stream dihentikan oleh pengguna.")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Bersihkan resources
        client.stop()
        if video_writer is not None:
            video_writer.release()

        print(f"Stream telah dihentikan. Total {frame_count} frame diproses.")

        # Download video jika disimpan
        if save_video and os.path.exists(output_path):
            print(f"Video disimpan ke: {output_path}")
            try:
                files.download(output_path)
                print("Video download dimulai...")
            except:
                print("Tidak dapat memulai download otomatis.")

def main():
    """Fungsi utama untuk menjalankan program dengan input user"""
    # Input URL stream
    default_url = "wss://cctv.villabs.id/streamer-jsmpeg/streamer/fatur1"
    url = input(f"Masukkan URL WebSocket stream CCTV (default: {default_url}): ")
    if not url:
        url = default_url

    # Opsi tambahan
    show_video = input("Tampilkan video? (y/n, default: y): ").lower() != 'n'
    save_video = input("Simpan video? (y/n, default: n): ").lower() == 'y'
    output_path = None

    if save_video:
        output_path = input("Nama file output (kosongkan untuk otomatis): ")
        if not output_path:
            output_path = None

    display_interval = int(input("Interval menampilkan frame (default: 5): ") or 5)
    max_frames = int(input("Jumlah maksimum frame (-1 untuk tanpa batas, default: 500): ") or 500)

    stream_wss_cctv(
        ws_url=url,
        show_video=show_video,
        save_video=save_video,
        output_path=output_path,
        max_frames=max_frames,
        display_interval=display_interval
    )

# Untuk menjalankan langsung dengan URL yang diberikan:
print("Untuk menjalankan dengan URL WebSocket CCTV yang diberikan:")
print("stream_wss_cctv('wss://cctv.villabs.id/streamer-jsmpeg/streamer/fatur1')")
print("Atau jalankan fungsi main() untuk input interaktif")

ModuleNotFoundError: No module named 'google.colab'